# Load Data

> Functions that load the data for the map.   

**Contents**

`Solomon Geospatial Data`
- `SolomonGeo`: A class that cleans the solomon islandas census and geography data
- `SolomonGeo.read_test`: Loads and transforms the test data
- `SolomonGeo.get_geojson`: Returns the geo_df as a geojson datatset

In [ ]:
#| default_exp load_data

In [ ]:
#| export
from nbdev.showdoc import *
import geopandas as gpd
import pandas as pd
from git import Repo
import json
from fastcore import *
from fastcore.basics import patch
from fastcore.test import *
import sys
import topojson as tp
import pickle
from urllib.request import urlopen
import boto3
from botocore.config import Config
from dotenv import load_dotenv
import os

load_dotenv()

True

In [ ]:
#| hide
repo = Repo('.', search_parent_directories=True)
fp = str(repo.working_tree_dir) + "/testData/"
const_df = pd.read_csv(fp + 'sol_census_' + 'all' + '_' + '2009' + '.csv')
# Check that the files exist using fastcore (both census and geo)

## Solomon Geospatial Data
> Load the geography and census data
### Geography Data
Solomons islands geography data is organised at the levels
 - adm0 - The country as as whole, Solomon Islands
 - adm1 - Also referred to as the province e.g. Honiara, Malaita
 - adm2 - The Consituency e.g. Central Honiara
 - adm3 - Ward, the smallest geography I am reporting. E.g. Cruz

### Census Data
 Solomon islands census data has been used from the 2009 and 2019 census. For the respective census:
2009
 - We have the total population in for each of the administration regions
2019
 - There is only data available down to the province level

In [ ]:
os.getenv("ENDPOINT_SOLDATA")

'https://s3.us-east-005.backblazeb2.com'

In [ ]:
env

{'USER': 'tom',
 'VSCODE_WSL_EXT_LOCATION': '/mnt/c/Users/thoma/.vscode/extensions/ms-vscode-remote.remote-wsl-0.81.8',
 'SHLVL': '2',
 'WT_PROFILE_ID': '{2c4de342-38b7-51cf-b940-2309a097f518}',
 'HOME': '/home/tom',
 'DBUS_SESSION_BUS_ADDRESS': 'unix:path=/run/user/1000/bus',
 'WSL_DISTRO_NAME': 'Ubuntu',
 'WAYLAND_DISPLAY': 'wayland-0',
 'LOGNAME': 'tom',
 'NAME': 'Code',
 'WSL_INTEROP': '/run/WSL/490_interop',
 'PULSE_SERVER': 'unix:/mnt/wslg/PulseServer',
 '_': '/home/tom/miniforge3/bin/python',
 'TERM': 'xterm-color',
 'PATH': '/home/tom/miniforge3/bin:/home/tom/.vscode-server/bin/1a5daa3a0231a0fbba4f14db7ec463cf99d7768e/bin/remote-cli:/home/tom/miniforge3/bin:/home/tom/miniforge3/condabin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/usr/lib/wsl/lib:/mnt/c/WINDOWS/system32:/mnt/c/WINDOWS:/mnt/c/WINDOWS/System32/Wbem:/mnt/c/WINDOWS/System32/WindowsPowerShell/v1.0:/mnt/c/WINDOWS/System32/OpenSSH:/mnt/c/Program Files/NVIDIA Corporation/NVI

In [ ]:
#| export
class SolomonGeo:
    # TODO work out how to format the attributes
    # Look at nbdev docs maybe?
    # TODO change all data to int?
    # TODO - should I make this a dataclass for the auto functionaliy? potentially should try it out
    '''
    Load the solomon islands geography data 
    Attributes:
        geo_df    Geopandas dataframe containing geographies and census data
        geo_levels    A list of the types of available aggregations
        census_vars    A dictionary of census variables in the dataset 
        data_type   Specifies whether the variable is a percentage or number
        locations A dictionary of locations accessed by the geography level
    '''
    def __init__(self, 
                geo_df:gpd.GeoDataFrame): # A geopandas dataset containing population and geography boundaries for each aggregation
        self.geo_df = geo_df

        # variable that tracks the types of aggregations
        self.geo_levels = geo_df.loc[:, ('core', 'agg')].unique()

        # Save a list of census variables, ignoring the core variables
        # Use a dictionary that maps the upper level column names to lower level ones
        var_df = geo_df.drop(columns = "core", level=0)
        vars = {}
        for col in var_df.columns:
            if col[0] not in vars:
                vars[col[0]] = [col[1]]
            else:
                vars[col[0]].append(col[1])
        self.census_vars = vars

        # TODO should captialise first letter
        self.data_type = geo_df.loc[:, ('core', 'type')].unique()

        # save a list of locations as a dictionary access by geography level
        locations = {}
        for geo in self.geo_levels:
            locations[geo] = geo_df.loc[geo_df['core']['agg'] == geo].index.unique().sort_values()
        self.locations = locations
        # TODO: need a list of column sub headings: get from column name split by `:`

        self.type_default = 'Total'


    @classmethod
    def read_test(cls,
                 ): # A solmon geo class TODO work out how to return self here... (can't?)
        '''
        Initialise the object from files using the local testing data
        '''

        repo = Repo('.', search_parent_directories=True)
        pw = str(repo.working_tree_dir) + "/testData/"
        df = pd.read_csv(pw + 'sol_census_all_2009.csv')
        aggs = df.loc[:, 'agg'].unique()
        geos = []
        for agg in aggs:
            geo = gpd.read_file(pw + 'sol_geo_' + agg.lower() + '.json')
            # Add an agg column, as the data and geometry need to be joined by id and agg
            geo.loc[:, 'agg'] = agg
            geos.append(geo)

        gdf = cls.transform(df, geos)
        return cls(
            geo_df = gdf
        )
    
    @classmethod
    def transform(cls, 
                    df:pd.DataFrame, # The dataframe containing census data
                    l_geos:[gpd.GeoDataFrame], # A list of geopandas dataframes containing 
                                                # the geographies 
                 ) -> gpd.GeoDataFrame: # Returns combined dataset
        '''
        Extract and return input datasets from file. Assumes correct format of input dataset, then
        Transform given raw input dataset into a cleaned and combined geopandas dataframe
        '''
        # TODO - if I add more years, loop per year

        geos = gpd.GeoDataFrame()
        for geo in l_geos:
            # Before combining, need to rename like columns
            # Rename columns and keep only necessary ones, Note that id can be province id, contsituency id etc.
            geo.columns = geo.columns.str.replace(r'^[a-zA-Z]+name$', 'geo_name', case = False, regex = True)
            # TODO this assumes the id key column is the first one (which so far it is...)
            geo.rename(columns = {geo.columns[0]:'id'}, inplace=True)

            geo = geo.loc[:, ['id', 'agg', 'geometry']] 

            # simplify the geography, use topo to preserver the topology between shapes
            topo = tp.Topology(geo, prequantize=False)
            geo = topo.toposimplify(360/43200).to_gdf()

            geos = pd.concat([geos, geo])
            
        # Clean the geospatial dataframe
        geos.loc[:, 'year'] = '2009'
        
        # Clean the census data
        df = df.dropna()
        # Rename columns to be consistent across geography
        df.columns = df.columns.str.replace(r'^[a-zA-Z]+_name$', 'geo_name', case = False, regex = True)
        # id needs to change types twice so that it is a string of an int
        df = df.astype({'id': 'int'})#, 'male_pop':'int', 	'female_pop':'int', 'total_pop':'int'})
        df = df.astype({'id': 'str'})
        
        # Merge the data together
        geo_df = geos.merge(df, on=['id', 'agg']).set_index("geo_name") # , 'geo_name'

        # Convert into a multiindex dataframe, with hiearchical columns
        try:
            geo_df = geo_df.rename(columns = {'geometry':'core: geometry', 
                                          'id':'core: id', 'agg':'core: agg', 
                                          'year':'core: year', 'type':'core: type'})
            cols = geo_df.columns.str.extract(r'(.*): (.+)', expand=True)
            geo_df.columns = pd.MultiIndex.from_arrays((cols[0], cols[1]))
            geo_df.columns.names = [None]*2
        except:
            raise ValueError("Issue converting geopandas dataframe to multindex. \
                             Check that all columns have \': \' beside the following\
                             core columns: geometry, id, agg, year, type.")
        # Turn the transformed dataset
        return geo_df


    @classmethod
    def load_pickle(cls,
                    folder:str, #file path of the folder to save in
                    b2:bool = False, # Whether to load from github or local
                    file_name:str = 'sol_geo.pickle' # file name of the saved class
                 ):
        '''
        Initialise the object from a saved filepath
        '''
        if b2:
            #print('https://raw.githubusercontent.com/Gippers/SolomonIslandsDataMap/main' + folder + file_name)
            #tmp_geo = pickle.load(urlopen('https://raw.githubusercontent.com/Gippers/SolomonIslandsDataMap/main' + folder + file_name))
            # Return a boto3 resouce object for b2 service
            b2 = boto3.resource(service_name='s3',
                        endpoint_url=os.getenv("ENDPOINT_SOLDATA"),   
                        aws_access_key_id=os.getenv("KEY_ID_SOLDATA"),              # Backblaze keyID
                        aws_secret_access_key=os.getenv("APPLICATION_KEY_SOLDATA"), # Backblaze applicationKey
                        config = Config(
                            signature_version='s3v4',
                    ))
            obj = b2.Bucket(os.getenv("KEY_NAME_SOLDATA")).Object("sol_geo.pickle").get()
            tmp_geo = obj['Body'].read()
            print(tmp_geo)
            # TODO getting closer keep looking here: https://stackoverflow.com/questions/36205481/read-file-content-from-s3-bucket-with-boto3

        else:
            # TODO work out how to make this a class method
            repo = Repo('.', search_parent_directories=True)
            pw = str(repo.working_tree_dir) + folder + file_name
            
            with open(pw, 'rb') as f:
                tmp_geo = pickle.load(f)
 
        
        return cls(
            geo_df = gpd.GeoDataFrame(tmp_geo['geo_df'])
        )
        

In [ ]:
#| hide
show_doc(SolomonGeo)

---

[source](https://github.com/Gippers/SolomonIslandsDataMap/blob/main/SolomonIslandsDataMap/load_data.py#L22){target="_blank" style="float:right; font-size:smaller"}

### SolomonGeo

>      SolomonGeo (geo_df:geopandas.geodataframe.GeoDataFrame)

Load the solomon islands geography data 
Attributes:
    geo_df    Geopandas dataframe containing geographies and census data
    geo_levels    A list of the types of available aggregations
    census_vars    A dictionary of census variables in the dataset 
    data_type   Specifies whether the variable is a percentage or number
    locations A dictionary of locations accessed by the geography level

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| geo_df | GeoDataFrame | A geopandas dataset containing population and geography boundaries for each aggregation |

In [ ]:
#| hide
show_doc(SolomonGeo.read_test)

---

[source](https://github.com/Gippers/SolomonIslandsDataMap/blob/main/SolomonIslandsDataMap/load_data.py#L68){target="_blank" style="float:right; font-size:smaller"}

### SolomonGeo.read_test

>      SolomonGeo.read_test ()

Initialise the object from files using the local testing data

## Save and Load

### Save Solomon Geo
Saves a solmon geo class in a pickle

In [ ]:
#| export
@patch
def save_pickle(self:SolomonGeo,
              folder:str, #file path of the folder to save in
                file_name:str = 'sol_geo.pickle' # file name of the saved class
             ):
    '''
    Save a pickle of the SolomonGeo class in backblaze b2
    '''
    repo = Repo('.', search_parent_directories=True)
    pw = str(repo.working_tree_dir) + folder + file_name
    
    f = open(pw, 'wb')
    pickle.dump(self.__dict__, f, 2)
    f.close()


In [ ]:
show_doc(SolomonGeo.save_pickle)

---

[source](https://github.com/Gippers/SolomonIslandsDataMap/blob/main/SolomonIslandsDataMap/load_data.py#L176){target="_blank" style="float:right; font-size:smaller"}

### SolomonGeo.save_pickle

>      SolomonGeo.save_pickle (folder:str, file_name:str='sol_geo.pickle')

Save a pickle of the SolomonGeo class in backblaze b2

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| folder | str |  | file path of the folder to save in |
| file_name | str | sol_geo.pickle | file name of the saved class |

In [ ]:
show_doc(SolomonGeo.load_pickle)

---

[source](https://github.com/Gippers/SolomonIslandsDataMap/blob/main/SolomonIslandsDataMap/load_data.py#L149){target="_blank" style="float:right; font-size:smaller"}

### SolomonGeo.load_pickle

>      SolomonGeo.load_pickle (folder:str, b2:bool=False,
>                              file_name:str='sol_geo.pickle')

Initialise the object from a saved filepath

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| folder | str |  | file path of the folder to save in |
| b2 | bool | False | Whether to load from github or local |
| file_name | str | sol_geo.pickle | file name of the saved class |

## Get Geo JSON
A getter method for the geometry portion of the dataset that returns a geoJson formated geography. 

It only includes the geography and location name as id

In [ ]:
#| export
@patch
def get_geojson(self:SolomonGeo, 
                geo_filter:str = None, # Filters the geojson to the requested aggregation 
               ) -> dict: # Geo JSON formatted dataset
    '''
    A getter method for the SolomonGeo class that returns a Geo JSON formatted dataset
    '''
    ret = self.geo_df
    # Only need geojson from one half of the dataset
    ret = ret.loc[ret['core']['type'] == self.type_default, :]
    if geo_filter is not None:
        ret = ret.loc[ret['core']['agg'] == geo_filter, :]
    # Return only the core data to minimise the html size
    return json.loads(ret.loc[:, ('core', 'geometry')].to_json())

In [ ]:
#| hide
show_doc(SolomonGeo.get_geojson)

---

[source](https://github.com/Gippers/SolomonIslandsDataMap/blob/main/SolomonIslandsDataMap/load_data.py#L193){target="_blank" style="float:right; font-size:smaller"}

### SolomonGeo.get_geojson

>      SolomonGeo.get_geojson (geo_filter:str=None)

A getter method for the SolomonGeo class that returns a Geo JSON formatted dataset

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| geo_filter | str | None | Filters the geojson to the requested aggregation |
| **Returns** | **dict** |  | **Geo JSON formatted dataset** |

In [ ]:
#| export
@patch
def get_df(self:SolomonGeo, 
                geo_filter:str = None, # Filters the dataframe to the requested geography 
                var:str = None, # Selects an upper level 
                measure:str = None, # Selects the lower level variable, if var 1 is used, measure must be used.
                loc_filter:[str] = None, # Filters one of more locations
                # TODO remove hardcoding here?
                type_filter:str = 'Total', # Return either number of proportion
                agg_flag = False, # Whether to return the dataset aggregated for the given selection
               ) -> pd.DataFrame: # Pandas Dataframe containing population data
    '''
    A getter method for the SolomonGeo class that returns a pandas dataset containg
    the id variable and the total population variable. This is the minimal data required
    to display on the map. 
    '''
    ret = self.geo_df
    ret = ret.loc[ret['core']['type'] == type_filter, :]
    # TODO check that filter is valid
    if geo_filter is not None:
        try:
            assert(geo_filter in ['Ward', 'Constituency', 'Province'])
        except:
            ValueError("Geo filter must be one of: ['Ward', 'Constituency', 'Province']")
        ret = ret.loc[ret['core']['agg'] == geo_filter, :]
    # Return only the core data to minimise the html size
    ret = ret.drop(columns = 'core', level=0)

    if loc_filter is not None:
        ret = ret.loc[ret.index.isin(loc_filter), :]

    # Keep only selected column if required
    if measure is not None:
        try:
            assert(var is not None)
            assert(measure in self.census_vars[var])
        except:
            ValueError("If measure is set, var 1 must be set and the key value pair of var and measure must match")
        ret = ret[var].filter(items = [measure])
    elif var is not None:
        # Keep all values from upper level column
        ret = ret[var]
        
    return pd.DataFrame(ret)

In [ ]:
#| hide
show_doc(SolomonGeo.get_df)

---

[source](https://github.com/Gippers/SolomonIslandsDataMap/blob/main/SolomonIslandsDataMap/load_data.py#L209){target="_blank" style="float:right; font-size:smaller"}

### SolomonGeo.get_df

>      SolomonGeo.get_df (geo_filter:str=None, var:str=None, measure:str=None,
>                         loc_filter:[<class'str'>]=None,
>                         type_filter:str='Total', agg_flag=False)

A getter method for the SolomonGeo class that returns a pandas dataset containg
the id variable and the total population variable. This is the minimal data required
to display on the map.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| geo_filter | str | None | Filters the dataframe to the requested geography |
| var | str | None | Selects an upper level |
| measure | str | None | Selects the lower level variable, if var 1 is used, measure must be used. |
| loc_filter | [<class 'str'>] | None | Filters one of more locations |
| type_filter | str | Total | Return either number of proportion |
| agg_flag | bool | False | Whether to return the dataset aggregated for the given selection |
| **Returns** | **DataFrame** |  | **Pandas Dataframe containing population data** |

## Aggregate Dataframe
Returns an aggregate pandas series 

In [ ]:
#| export
@patch
def agg_df(self:SolomonGeo, 
                geo_filter:str = None, # Filters the dataframe to the requested geography 
                var:str = None, # Selects an upper level 
                measure:str = None, # Selects the lower level variable, if var 1 is used, measure must be used.
                loc_filter:[str] = None, # Filters one of more locations
                # TODO remove hardcoding here?
                type_filter:str = 'Total', # Return either number of proportion
               ) -> pd.Series: # Pandas data series containing aggregated and filtered data
    '''
    A getter method for the SolomonGeo class that calls get_df to get a spcific and then further 
    aggregates that dataset so that the proportion is the weighted proportion
    '''
    df = self.get_df(geo_filter, var, measure, loc_filter)

    if type_filter == 'Total':
        df = df.sum()
    elif type_filter == 'Proportion':
        df = df.sum() / df.sum().sum() * 100
    else:
        raise ValueError('The type passed to the aggregate function must be one of the following: \'Total\', \'Proportion\'.')
    return df

In [ ]:
show_doc(agg_df)

---

[source](https://github.com/fastai/nbdev/blob/master/nbdev/showdoc.py#LNone){target="_blank" style="float:right; font-size:smaller"}

### show_doc

>      show_doc (sym, renderer=None, name:str|None=None, title_level:int=3)

Show signature and docstring for `sym`

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| sym |  |  | Symbol to document |
| renderer | NoneType | None | Optional renderer (defaults to markdown) |
| name | str \| None | None | Optionally override displayed name of `sym` |
| title_level | int | 3 | Heading level to use for symbol name |

# Testing

In [ ]:
sol_geo = SolomonGeo.read_test()

In [ ]:
sol_geo.geo_df

core                 \
                                                   geometry   id       agg   
geo_name                                                                     
Choiseul  MULTIPOLYGON (((157.56463 -7.49272, 157.56005 ...    1  Province   
Choiseul  MULTIPOLYGON (((157.56463 -7.49272, 157.56005 ...    1  Province   
Honiara   MULTIPOLYGON (((159.91806 -9.42379, 159.92354 ...   10  Province   
Honiara   MULTIPOLYGON (((159.91806 -9.42379, 159.92354 ...   10  Province   
Western   MULTIPOLYGON (((158.25148 -8.78861, 158.24885 ...    2  Province   
...                                                     ...  ...       ...   
Vanikoro  MULTIPOLYGON (((166.98326 -11.67945, 166.98258...  915      Ward   
Tikopia   MULTIPOLYGON (((168.84036 -12.28504, 168.82691...  916      Ward   
Tikopia   MULTIPOLYGON (((168.84036 -12.28504, 168.82691...  916      Ward   
Neo       MULTIPOLYGON (((165.81001 -10.65326, 165.79693...  917      Ward   
Neo       MULTIPOLYGON (((165.81001 -10.65326, 165.79693...  917      Ward   

                             Key Statistics  \
          year        type Total Households   
geo_name                                      
Choiseul  2009       Total             4712   
Choiseul  2009  Proportion              100   
Honiara   2009       Total             8981   
Honiara   2009  Proportion              100   
Western   2009       Total            13762   
...        ...         ...              ...   
Vanikoro  2009  Proportion              100   
Tikopia   2009       Total              262   
Tikopia   2009  Proportion              100   
Neo       2009       Total              301   
Neo       2009  Proportion              100   

         Main source of household drinking water                     \
                     metered SIWA drinking water communal standpipe   
geo_name                                                              
Choiseul                                    2.00            1416.00   
Choiseul                                    0.04              30.05   
Honiara                                  6771.00             351.00   
Honiara                                    75.39               3.91   
Western                                    37.00            4151.00   
...                                          ...                ...   
Vanikoro                                    0.38              62.78   
Tikopia                                     0.00             255.00   
Tikopia                                     0.00              97.33   
Neo                                         0.00               1.00   
Neo                                         0.00               0.33   

                                                 ...  \
         private water tank communal water tank  ...   
geo_name                                         ...   
Choiseul            1137.00              840.00  ...   
Choiseul              24.13               17.83  ...   
Honiara              861.00              134.00  ...   
Honiara                9.59                1.49  ...   
Western             4596.00             2426.00  ...   
...                     ...                 ...  ...   
Vanikoro               3.38                1.50  ...   
Tikopia                1.00                4.00  ...   
Tikopia                0.38                1.53  ...   
Neo                   22.00              210.00  ...   
Neo                    7.31               69.77  ...   

         Household money received from remittances  \
                          1000 - 1499 S.I. dollars   
geo_name                                             
Choiseul                                    209.00   
Choiseul                                      4.44   
Honiara                                     203.00   
Honiara                                       2.26   
Western                                     530.00   
...                                            ...   
Vanikoro                             

In [ ]:
sol_geo.locations

{'Province': Index(['Central', 'Choiseul', 'Guadalcanal', 'Honiara', 'Isabel',
        'Makira-Ulawa', 'Malaita', 'Rennell-Bell', 'Temotu', 'Western'],
       dtype='object', name='geo_name'),
 'Constituency': Index(['Auki-Langalanga', 'Baegu-Asifola', 'Central Guadalcanal',
        'Central Honiara', 'Central Kwara'ae', 'Central Makira', 'East AreAre',
        'East Choiseul', 'East Guadalcanal', 'East Honiara', 'East Kwaio',
        'East Makira', 'East Malaita', 'East-Central', 'Fataleka', 'Gao-Bugotu',
        'Gizo-Kolombangara', 'Hograno-Kia-Havulei', 'Lau-Mbaelela',
        'Malaita Outer Island', 'Marigne-Kokota', 'Marovo', 'Nggela',
        'North Guadalcanal', 'North Malaita', 'North New Georgia',
        'North Vella la Vella', 'North West Choiseul', 'North West Guadalcanl',
        'North-East', 'Ranongga-Simbo', 'Rennell-Bellona',
        'Savo-Russull Islands', 'Shortland Islands', 'Small Malaita',
        'South Choiseul', 'South Guadalcanal', 'South New Georgia-Rendova'

Save and Load

In [ ]:
sol_geo.save_pickle('/testData/')

In [ ]:
SolomonGeo.load_pickle('/testData/', b2=True)

Test filtering of down to multiple locations

In [ ]:
#test_eq(sol_geo.get_df(geo_filter = 'constituency')['total_pop'].sum(), const_df['total_pop'].sum())
sol_geo.get_df(geo_filter = 'Constituency', loc_filter=['Nggela', 'East AreAre'])

Key Statistics Main source of household drinking water  \
            Total Households             metered SIWA drinking water   
geo_name                                                               
East AreAre             1105                                     1.0   
Nggela                  3315                                    26.0   

                                                                       \
            communal standpipe private water tank communal water tank   
geo_name                                                                
East AreAre              815.0               12.0                 9.0   
Nggela                  1557.0              258.0               437.0   

                                                                               \
            well - protected well - unprotected river or stream bottled water   
geo_name                                                                        
East AreAre              1.0                3.0           223.0           0.0   
Nggela                  52.0               13.0           868.0           1.0   

                                            ...  \
            other source of drinking water  ...   
geo_name                                    ...   
East AreAre                           41.0  ...   
Nggela                               103.0  ...   

            Household money received from remittances  \
                             1000 - 1499 S.I. dollars   
geo_name                                                
East AreAre                                      41.0   
Nggela                                           25.0   

                                         \
            more than 1500 S.I. dollars   
geo_name                                  
East AreAre                        15.0   
Nggela                             59.0   

            Main source of household income in last 12 months               \
                                                    No income Wages Salary   
geo_name                                                                     
East AreAre                                              37.0         89.0   
Nggela                                                   49.0        558.0   

                                                                     \
            Own business Sale fish crop craft Land lease House rent   
geo_name                                                              
East AreAre        101.0                650.0        0.0        2.0   
Nggela             327.0               1878.0        0.0        4.0   

                                      
            Remittances Other source  
geo_name                              
East AreAre        31.0        195.0  
Nggela             75.0        424.0  

[2 rows x 66 columns]

In [ ]:
# TODO work out how to check save and load are the same as a test
# Might need to wreite and equality function

In [ ]:
# TODO should do a count check for the geojson similar to this
sol_geo.geo_df[sol_geo.geo_df['core']['agg'] == 'Constituency'].count()

core                                               geometry                100
                                                   id                      100
                                                   agg                     100
                                                   year                    100
                                                   type                    100
                                                                          ... 
Main source of household income in last 12 months  Sale fish crop craft    100
                                                   Land lease              100
                                                   House rent              100
                                                   Remittances             100
                                                   Other source            100
Length: 71, dtype: int64

# TODO check that all proportions are less than 0

In [ ]:
# TODO filter and test equality?
test = sol_geo.get_geojson(geo_filter = 'ward')

The sum after merging and filtering should equal the sum from the raw dataset.

In [ ]:
test_eq(sol_geo.agg_df(geo_filter = 'Constituency',var = 'Key Statistics', measure = 'Total Households')[0], 
        const_df.loc[const_df['type'] == 'Total'].loc[const_df['agg'] == 'Constituency']['Key Statistics: Total Households'].sum())

/tmp/ipykernel_5384/1266885615.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  test_eq(sol_geo.agg_df(geo_filter = 'Constituency',var = 'Key Statistics', measure = 'Total Households')[0],


In [ ]:
sol_geo.agg_df(geo_filter = 'Constituency', var='Key Statistics', measure = 'Total Households')

Total Households    91251
dtype: int64

In [ ]:
sol_geo.get_df(geo_filter = 'Constituency', var='Key Statistics', measure = 'Total Households', loc_filter=['West Guadalcanal']).values[0]

array([1671])

In [ ]:
sol_geo.geo_df['core'].filter(items = ['id'])

,id
geo_name,
Choiseul,1
Choiseul,1
Honiara,10
Honiara,10
Western,2
...,...
Vanikoro,915
Tikopia,916
Tikopia,916


In [ ]:
# TODO -test that they all sum to one
test_agg = sol_geo.agg_df(geo_filter = 'Constituency', type_filter='Proportion', var = "Main source of household drinking water")
test_eq(test_agg.sum(), 100.0)
test_agg

metered SIWA drinking water        9.168119
communal standpipe                35.114136
private water tank                12.471096
communal water tank               10.619062
well - protected                   2.081073
well - unprotected                 1.781898
river or stream                   24.543293
bottled water                      0.611500
other source of drinking water     3.609823
dtype: float64

In [ ]:
test_agg.sum()

100.0

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()